In [ ]:
# !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
# pip install python-dotenv

In [2]:
import minsearch
import json

In [3]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

# docs_raw[0]

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

# documents[0]

In [5]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [60]:
q='the course has already started, can I still enroll?'

In [61]:
index.fit(documents)

In [62]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5)

In [63]:
from openai import OpenAI

In [64]:
client = OpenAI()

In [66]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"It's possible you may still be able to enroll in the course, but it depends on the policies of the institution or the specific course you're interested in. Here are some steps you can take:\n\n1. **Check the Enrollment Deadline:** Look at the course details or contact the admissions office to find out the official deadline for enrollment.\n\n2. **Contact the Instructor:** Reach out to the course instructor directly to explain your situation and inquire if late enrollment is an option.\n\n3. **Advising Office:** Visit or contact your academic advisor or the department offering the course to discuss your options.\n\n4. **Online Portals:** If the course is offered online, there might be an option to enroll through the school's online platform. Check any notifications or messages related to course enrollment.\n\n5. **Special Permissions:** Sometimes, departments or instructors can give special permission for late enrollment, particularly if you have a valid reason and the course isn't too

In [52]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [69]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.+
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    """.strip()
    
    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [54]:
def llm(prompt):
    response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [70]:
query = 'How do I run kafka?'

def rag(query):
    search_result = search(query)
    prompt = build_prompt(query, search_result)
    answer = llm(prompt)

    return answer

In [72]:
print(rag(query))

To run Kafka, you can follow the instructions below based on whether you're working with Java or Python:

### For Java:

Navigate to your project directory and run the following command:
```sh
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java
```
Make sure to replace `<jar_name>` with the actual name of your jar file.

### For Python:

1. First, create and activate a virtual environment. In your terminal, run:
    ```sh
    python -m venv env
    source env/bin/activate   # MacOS/Linux
    # For Windows, use:
    # env\Scripts\activate
    ```

2. Install the necessary packages by running:
    ```sh
    pip install -r ../requirements.txt
    ```

3. To deactivate the virtual environment, run:
    ```sh
    deactivate
    ```

Ensure that your Kafka Docker images are up and running before executing these steps. These instructions should work on MacOS, Linux, and Windows (with slight path differences for activation).


In [73]:
from elasticsearch import Elasticsearch

In [74]:
es_client = Elasticsearch("http://localhost:9200")

In [75]:
es_client.info()

ObjectApiResponse({'name': '0fd2987cabf9', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'i4Dw5aS8Qp2GmJqjBsQo8Q', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [76]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [77]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [79]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████████████| 948/948 [00:21<00:00, 43.79it/s]


In [80]:
query = 'I just disovered the course. Can I still join it?'

In [93]:
def elastic_search(query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    
    index_name = "course-questions"
    
    response = es_client.search(index=index_name, body=search_query)

    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [94]:
elastic_search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta

In [96]:
def rag(query):
    search_result = elastic_search(query)
    prompt = build_prompt(query, search_result)
    answer = llm(prompt)

    return answer

In [97]:
rag(query) 

"Yes, you can still join the course even after it has started. You are eligible to submit homework, but be mindful of the deadlines for the final projects. Don't leave everything for the last minute."